In [1]:
# Load library
import csv
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta

In [ ]:
data = pd.read_csv("data/international-football-results-from-1872-to-2017.csv")
data.head()


In [8]:
# Not important
import sqlite3
import pandas.io.sql as sql
con = sqlite3.connect('database.sqlite')
# table = sql.read_frame('select * from some_table', con)
table = sql.read_sql('select * from football_data', con)
table.to_csv('output_1.csv')
# table.head()
table.to_csv('output.csv')


In [125]:
# Put everything together
# 1. Read data:

# 2. Add new column named "id" and "winner"
data['id'] = data.index
data['winner'] = np.where(data.home_score > data.away_score, data.home_team, 
                                     np.where(data.home_score < data.away_score, data.away_team, 'None'))
# filter data from 2005
data_2005 = data[data['date'] > '2005-01-01'].copy()
# data_2005 = data[data['date'] > '2016-01-01'].copy()
# define my own dataframe
col_names = ['id','date', 'team_1', 'team_2', 'home_team', 'tournament', 'h_win_diff', 'h_draw', 'f_goalF_1', 
             'f_goalF_2','f_goalA_1','f_goalA_2', 'f_win_1', 'f_win_2', 'f_draw_1', 'f_draw_2','ratio_odds','avg_odds_draw','result']
my_df = pd.DataFrame(columns = col_names)

start_time = time.time()
data_len = data_2005.shape[0]
for i in range(data_len):
#     print(i)
    # Current Match
    curr_match = data_2005.iloc[i,:]
    
    # Get the head-to-head result  
    pre_df = curr_match[['id','date','tournament']].copy() #Initialize dataframe
    pre_df['team_1'] =  curr_match['home_team'] if curr_match['home_team'] < curr_match['away_team'] else curr_match['away_team'] 
    pre_df['team_2'] =  curr_match['away_team'] if curr_match['home_team'] < curr_match['away_team'] else curr_match['home_team'] 
    pre_df['home_team'] = curr_match['home_team']
    
    team_1 = pre_df['team_1']
    team_2 = pre_df['team_2']
    
    df_h2h = data[(((data.home_team == team_1) & (data.away_team == team_2)) | 
                  ((data.home_team == team_2) & (data.away_team == team_1))) & 
                  (data.date < curr_match.date)]


    count = Counter(df_h2h['winner']) #Now we get no.win and no.draw   
    
    pre_df['h_win_diff'] = count[team_1] - count[team_2]
    pre_df['h_draw'] = count['None']
    
    
    # Get the recent result of each team
    # Team 1
#     team_1 = pre_df['team_1']
    df_10  = data[((data.home_team == team_1) | (data.away_team == team_1)) & 
                 (data.date < curr_match.date)]
    
    length = df_10.shape[0]
    df_10  = df_10.iloc[length-11:length-1,:]
    
    df_10['winner'] = np.where(df_10.home_score > df_10.away_score, df_10.home_team, 
                                     np.where(df_10.home_score < df_10.away_score, df_10.away_team, 'None'))
    
    pre_df['f_goalF_1'] = df_10['home_score'].sum()
    pre_df['f_goalA_1'] = df_10['away_score'].sum()
    
    dict_result_10 = Counter(df_10['winner'])
    pre_df['f_win_1']  = dict_result_10[team_1]
    pre_df['f_draw_1'] = dict_result_10['None']
    
    # Team 2
#     team_2 = pre_df['team_2']
    df_10  = data[((data.home_team == team_2) | (data.away_team == team_2)) & 
                 (data.date < curr_match.date)]
    
    length = df_10.shape[0]
    df_10  = df_10.iloc[length-11:length-1,:]
    
    df_10['winner'] = np.where(df_10.home_score > df_10.away_score, df_10.home_team, 
                                     np.where(df_10.home_score < df_10.away_score, df_10.away_team, 'None'))
    
    pre_df['f_goalF_2'] = df_10['home_score'].sum()
    pre_df['f_goalA_2'] = df_10['away_score'].sum()
    
    dict_result_10 = Counter(df_10['winner'])
    pre_df['f_win_2']  = dict_result_10[team_2]
    pre_df['f_draw_2'] = dict_result_10['None']
    
    pre_df['result'] = np.where(curr_match.winner == team_1, 'win', 
                                     np.where(curr_match.winner == team_2, 'lose', 'draw'))
    
    
    my_df = my_df.append(pre_df)
    
end_time = time.time()
# print(time.time() - start_time)
print('Execution Time : {}'.format(end_time - start_time))
my_df.to_csv('data/data_2005.csv')

Execution Time : 1371.9029848575592


In [124]:
#  Save the file to csv
my_df.to_csv('data/data_2016.csv')

In [2]:
# Put data with bet odd and data without odd

# Load data with bet
data_odd = pd.read_csv("data/data_odd_match_filter.csv", encoding='utf-8')

# load data without bet
data_wo_odd = pd.read_csv("data/data_2005.csv", encoding='utf-8')

data_wo_odd['date'] = pd.to_datetime(data_wo_odd['date'])
data_odd['match_date'] = pd.to_datetime(data_odd['match_date'])

In [3]:
# Because we end up only use data_odd, we need lookup match data from data_2005. How?
# First, look for same date, Second look for the team

# Create new dataframe
# define my own dataframe
my_df = data_wo_odd.iloc[2:3,:].copy()
my_df = my_df[0:0] #erase data but keep schema

# from datetime import datetime, timedelta


# data_wo_odd['date'] = pd.to_datetime(data_wo_odd['date'])
# data_odd['match_date'] = pd.to_datetime(data_odd['match_date'])

data_len = data_odd.shape[0]

for i in range(data_len):
# for i in range(70,76):
    print(i)
    curr_match = data_odd.iloc[i,:]
    
    # Lookup logic
    # match the date -> match name of 2 teams
    # if couldn't find -> shift day by -1 or +1

#     curr_date = curr_match.match_date

#     print(datetime.strptime(curr_match.match_date, '%m/%d/%Y').date())
#     curr_date = datetime.strptime(curr_match.match_date, '%m/%d/%Y').date()
#     look_date = datetime.strptime(data_wo_odd.date, '%m/%d/%Y').date()
#     if (curr_date == look_date):
#         lookup_match = data_wo_odd[((data_wo_odd.team_1 == curr_match.home_team) & 
#                                    (data_wo_odd.team_2 == curr_match.away_team)) |
#                                   ((data_wo_odd.team_1 == curr_match.away_team) & 
#                                    (data_wo_odd.team_2 == curr_match.home_team))]
    
    lookup_match = data_wo_odd[(data_wo_odd.date == curr_match.match_date) &
                              (((data_wo_odd.team_1 == curr_match.home_team) & 
                               (data_wo_odd.team_2 == curr_match.away_team)) |
                              ((data_wo_odd.team_1 == curr_match.away_team) & 
                               (data_wo_odd.team_2 == curr_match.home_team)))]
    if (lookup_match.shape[0] == 0):#Wrong date -> shift date - 1

        lookup_match = data_wo_odd[(data_wo_odd.date == curr_match.match_date + timedelta(days=-1)) &
                                  (((data_wo_odd.team_1 == curr_match.home_team) & 
                                   (data_wo_odd.team_2 == curr_match.away_team)) |
                                  ((data_wo_odd.team_1 == curr_match.away_team) & 
                                   (data_wo_odd.team_2 == curr_match.home_team)))] 
        
        if (lookup_match.shape[0] == 0): #Wrong date -> shift date + 1
            lookup_match = data_wo_odd[(data_wo_odd.date == curr_match.match_date + timedelta(days=+1)) &
                                      (((data_wo_odd.team_1 == curr_match.home_team) & 
                                       (data_wo_odd.team_2 == curr_match.away_team)) |
                                      ((data_wo_odd.team_1 == curr_match.away_team) & 
                                       (data_wo_odd.team_2 == curr_match.home_team)))]   
            if (lookup_match.shape[0] == 0):
#                 print('date mismatch')
    #             print(curr_match)
    
                # Something Wrong here, print them out
                print(curr_match.home_team + " - " + curr_match.away_team)
#             else:
#                 avg_odds_win_1 = 0
#                 avg_odds_win_2 = 0
#                 avg_odds_draw = 0
#                 print(lookup_match.team_1)
#                 print(curr_match.home_team)
                
                
#                 if (lookup_match.team_1 == curr_match.home_team):
#                     avg_odds_win_1 = curr_match.avg_odds_home_win
#                     avg_odds_win_2 = curr_match.avg_odds_away_win
#                 else:
#                     avg_odds_win_1 = curr_match.avg_odds_away_win
#                     avg_odds_win_2 = curr_match.avg_odds_home_win    

#                 pre_df = lookup_match.copy() #Initialize dataframe
#                 pre_df['avg_odds_draw'] = curr_match['avg_odds_draw']
#                 pre_df['avg_odds_win_1'] = avg_odds_win_1
#                 pre_df['avg_odds_win_2'] = avg_odds_win_2
                
#                 my_df = my_df.append(pre_df)

#     Fill in missing data
    
#     print(lookup_match['team_1'] + " - " + curr_match['home_team'])
    if(lookup_match.shape[0] != 0):
        lookup_match = lookup_match.squeeze()
#         print(type(lookup_match))
    #     print(type(curr_match))
    #     print(type(lookup_match['team_1']))
    #     print(type(curr_match['home_team']))


        if (lookup_match.team_1 == curr_match.home_team):
            avg_odds_win_1 = curr_match.avg_odds_home_win
            avg_odds_win_2 = curr_match.avg_odds_away_win
        else:
            avg_odds_win_1 = curr_match.avg_odds_away_win
            avg_odds_win_2 = curr_match.avg_odds_home_win    

        pre_df = lookup_match.copy() #Initialize dataframe
        pre_df['avg_odds_draw'] = curr_match['avg_odds_draw']
        pre_df['avg_odds_win_1'] = avg_odds_win_1
        pre_df['avg_odds_win_2'] = avg_odds_win_2

        my_df = my_df.append(pre_df)

my_df.to_csv('data/data_odd_2005.csv')      

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
Romania - Moldova
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272


1642
1643
1644
1645
1646
1647
1648
1649
1650
1651
1652
1653
1654
1655
1656
1657
1658
1659
1660
1661
1662
1663
1664
1665
1666
1667
1668
1669
1670
1671
1672
1673
Haiti - Salvador
1674
1675
1676
1677
1678
1679
1680
1681
1682
1683
1684
1685
1686
1687
1688
1689
1690
1691
1692
1693
1694
1695
1696
1697
1698
1699
1700
1701
1702
1703
1704
1705
1706
1707
1708
1709
1710
1711
1712
1713
1714
1715
1716
1717
1718
1719
Qatar - South Korea
1720
1721
1722
1723
1724
1725
1726
1727
1728
1729
1730
1731
1732
1733
1734
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1752
1753
1754
1755
1756
1757
1758
1759
1760
1761
1762
Singapore - Vietnam
1763
1764
Indonesia - Myanmar
1765
1766
1767
Singapore - Myanmar
1768
1769
1770
1771
Myanmar - Cambodia
1772
1773
1774
1775
1776
1777
1778
Thailand - Indonesia
1779
Singapore - Vietnam
1780
Oman - Senegal
1781
1782
1783
1784
1785
1786
Qatar - Libya
1787
1788
1789
1790
1791
1792
Kenya - Eritrea
1793
1794
1795
1796
1797
Burundi - Eritrea


2976
2977
2978
2979
2980
2981
2982
2983
2984
2985
2986
2987
2988
2989
2990
2991
2992
2993
2994
2995
2996
2997
Belize - Salvador
2998
2999
3000
3001
3002
3003
3004
3005
3006
3007
3008
3009
3010
Panama - Salvador
3011
3012
Netherlands - Norway
3013
3014
3015
3016
3017
Honduras - Salvador
3018
South Africa - Sweden
3019
3020
3021
3022
3023
3024
3025
3026
3027
Sudan - Uganda
3028
3029
3030
3031
3032
3033
3034
South Africa - Niger
3035
Ghana - Zimbabwe
3036
3037
3038
3039
3040
3041
3042
3043
3044
3045
3046
3047
3048
3049
3050
3051
3052
3053
3054
3055
3056
3057
3058
3059
3060
3061
3062
3063
3064
3065
3066
Cameroon - Mali
3067
3068
3069
Rwanda - Tunisia
3070
Senegal - Angola
3071
Gabon - Uganda
3072
Sudan - Algeria
3073
Ghana - Niger
3074
South Africa - Zimbabwe
3075
Cameroon - Ivory Coast
3076
3077
Rwanda - Angola
3078
Senegal - Tunisia
3079
South Africa - Algeria
3080
Sudan - Niger
3081
Cameroon - Angola
3082
Tunisia - Congo DR
3083
Algeria - Tunisia
3084
Sudan - Angola
3085
Algeria - Sudan

3989
3990
3991
3992
3993
3994
3995
3996
3997
3998
3999
4000
4001
Costa Rica - Salvador
4002
4003
4004
4005
4006
4007
4008
4009
4010
4011
4012
4013
4014
Puerto Rico - Tampa Bay
4015
Ethiopia - Central Africa
4016
4017
4018
4019
4020
4021
4022
4023
4024
4025
4026
4027
4028
4029
4030
4031
4032
4033
4034
4035
4036
4037
4038
4039
4040
4041
4042
4043
4044
Egypt - Central Africa
4045
4046
4047
4048
4049
4050
4051
Cameroon - Guinea Bissau
4052
4053
4054
4055
Mozambique - Tanzania
4056
4057
4058
4059
4060
4061
Puerto Rico - San Antonio
4062
4063
4064
4065
4066
4067
4068
4069
4070
4071
4072
Egypt - Sudan
4073
4074
4075
4076
4077
4078
Iraq - Egypt
4079
4080
4081
4082
4083
Egypt - Lebanon
4084
4085
4086
4087
4088
4089
4090
Puerto Rico - Edmonton
4091
Puerto Rico - San Antonio
4092
Mexico - Japan
4093
United Arab Emirates - Uruguay Ol.
4094
Puerto Rico - Minnesota
4095
Gibraltar - Notts County
4096
4097
4098
4099
4100
4101
4102
4103
4104
4105
4106
4107
4108
4109
4110
4111
4112
4113
4114
4115
4116
4

5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
Honduras - Venezuela
5300
Mexico - Nigeria
5301
Costa Rica - Paraguay
5302
USA - Mexico
5303
5304
5305
Madagascar - Uganda
5306
Kenya - Comoros
5307
Mozambique - South Sudan
5308
Swaziland - Sierra Leone
5309
5310
5311
5312
Maldives - Myanmar
5313
5314
5315
Myanmar - Palestine
5316
5317
5318
5319
5320
5321
5322
Kyrgyzstan - Myanmar
5323
5324
5325
5326
Morocco - Mozambique
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
USA - Azerbaijan
5345
5346
5347
5348
5349
5350
Mexico - Israel
5351
5352
5353
5354
5355
5356
5357
Hondura

In [27]:
# from iso3166 import countries
length = data_odd.shape[0]
# count = 0

# Initialze my_df
my_df = data_odd.iloc[1:3,:]
my_df = my_df[0:0]

for i in range(length):
    curr_match = data_odd.iloc[i,:]
    if curr_match['home_team'] in countries:
#         count = count + 1
        print(curr_match['home_team'])
        my_df = my_df.append(curr_match)
# print("Count: "+str(count))

Monaco
Monaco
Monaco
Latvia
Japan
Uzbekistan
Bahrain
Greece
Morocco
Spain
Hong Kong
Bulgaria
Algeria
Azerbaijan
Georgia
Slovakia
United Arab Emirates
Slovenia
Israel
Egypt
Malta
South Africa
Poland
Ireland
Germany
France
Italy
Monaco
Monaco
Monaco
Monaco
Kuwait
Saudi Arabia
Armenia
Bulgaria
Estonia
Liechtenstein
Croatia
Poland
Georgia
Israel
Romania
Turkey
Denmark
Belgium
Morocco
France
Italy
Australia
Slovenia
Portugal
Spain
Chile
Costa Rica
Guatemala
Mexico
Ireland
Japan
Andorra
Estonia
Croatia
Latvia
Ukraine
Georgia
Kuwait
Slovakia
Poland
Slovenia
Austria
Greece
Netherlands
San Marino
Switzerland
Hungary
Israel
Italy
USA
Panama
Monaco
Estonia
Monaco
Mexico
Monaco
Ecuador
Monaco
Monaco
Romania
Norway
Japan
Monaco
USA
Poland
France
Colombia
Finland
Uzbekistan
Bahrain
Saudi Arabia
Armenia
Azerbaijan
Bulgaria
Estonia
Sweden
Faroe Islands
Ukraine
Belarus
Turkey
Iceland
Albania
Ireland
Netherlands
Norway
San Marino
Portugal
Spain
Colombia
Ecuador
USA
Chile
Guatemala
Zimbabwe
Rwanda
Angola

Paraguay
Monaco
Bahrain
Chile
Hungary
Georgia
Luxembourg
Belarus
Latvia
Malta
San Marino
Serbia
Croatia
Netherlands
Slovakia
Sweden
Italy
Iceland
Portugal
Ghana
Cyprus
Ecuador
Peru
USA
Australia
Saudi Arabia
Switzerland
Austria
Kazakhstan
Andorra
Finland
Slovakia
Cyprus
Lithuania
Norway
Bulgaria
Turkey
Denmark
Slovenia
Iceland
Albania
Ukraine
France
Portugal
Spain
South Africa
Montenegro
Malta
Germany
Brazil
Canada
Colombia
Honduras
Peru
Monaco
Monaco
Bangladesh
Thailand
Oman
Lebanon
Palestine
Yemen
Cambodia
Slovakia
Hungary
Armenia
Belarus
Faroe Islands
Azerbaijan
Iceland
Cyprus
Liechtenstein
Romania
Denmark
Croatia
Greece
Poland
Slovenia
Belgium
Ireland
Italy
Uzbekistan
Uruguay
Argentina
Switzerland
Ecuador
Peru
Madagascar
Colombia
Mexico
Honduras
Croatia
Qatar
Kazakhstan
Azerbaijan
Ukraine
Georgia
Malta
Turkey
Denmark
Liechtenstein
Luxembourg
San Marino
Ireland
Netherlands
Sweden
Albania
Belgium
Germany
France
Sierra Leone
Japan
Estonia
United Arab Emirates
Poland
Finland
Austria
Sw

Japan
Kuwait
Hong Kong
China
Malaysia
Thailand
Honduras
Denmark
Kuwait
Egypt
Costa Rica
Belize
Monaco
San Marino
Honduras
USA
Guatemala
Nicaragua
Honduras
Singapore
Thailand
Indonesia
Yemen
Kuwait
Lebanon
United Arab Emirates
Bahrain
Panama
Mexico
Nicaragua
Costa Rica
Monaco
Honduras
Uzbekistan
Honduras
Costa Rica
Japan
Bahrain
Lithuania
Cyprus
Slovakia
Cyprus
Ukraine
Brazil
Ecuador
Cyprus
Serbia
Japan
Uzbekistan
Malta
San Marino
Ireland
Latvia
Mozambique
Libya
Angola
Iceland
Algeria
Cyprus
Estonia
Israel
Kuwait
Serbia
Egypt
Lithuania
Burkina Faso
Cameroon
South Africa
Turkey
Romania
Morocco
Greece
Austria
Germany
Switzerland
Tunisia
Belgium
Nigeria
France
Portugal
Spain
USA
Costa Rica
Peru
Colombia
Monaco
San Marino
Monaco
San Marino
Australia
Mexico
Monaco
San Marino
Qatar
Namibia
Uganda
Iceland
Saudi Arabia
Bahrain
Kuwait
Angola
Japan
Uzbekistan
Kenya
Rwanda
Armenia
Luxembourg
Togo
Sudan
Burkina Faso
Cyprus
Israel
Romania
Germany
Malta
Albania
Belgium
Ireland
Lithuania
Montenegro
Ne

Paraguay
Panama
Puerto Rico
Puerto Rico
Germany
Puerto Rico
San Marino
Israel
Netherlands
Algeria
Armenia
Kazakhstan
Andorra
Faroe Islands
Montenegro
Latvia
Romania
Sweden
Lithuania
Luxembourg
Estonia
Slovakia
Belgium
Greece
Liechtenstein
San Marino
Slovenia
France
Iceland
Portugal
China
Switzerland
Qatar
Kuwait
Uganda
Botswana
Gambia
South Africa
Sudan
Tunisia
Panama
Japan
Thailand
Poland
Canada
Morocco
Ethiopia
Mozambique
Swaziland
Zambia
Benin
Nigeria
Liberia
Egypt
Guatemala
Mexico
Jamaica
Georgia
Belarus
Bulgaria
Malta
Sweden
Turkey
Albania
Denmark
Austria
Croatia
Hungary
Netherlands
Norway
Serbia
Germany
Ireland
Switzerland
Italy
Japan
China
Estonia
Ukraine
United Arab Emirates
Poland
Qatar
Argentina
Canada
Jamaica
Mexico
Panama
India
Puerto Rico
Morocco
San Marino
Puerto Rico
Puerto Rico
Iraq
Monaco
Puerto Rico
San Marino
India
Armenia
Kazakhstan
Andorra
Georgia
Hungary
Cyprus
Luxembourg
Albania
Austria
Montenegro
Serbia
Germany
Greece
Ireland
Slovenia
Portugal
Spain
Puerto Rico


United Arab Emirates
Grenada
Saudi Arabia
Argentina
Monaco
Costa Rica
Ecuador
Brazil
Mexico
San Marino
Cuba
Mexico
Brazil
Uruguay
Mexico
Argentina
Uruguay
Argentina
San Marino
Kuwait
Qatar
Monaco
Gabon
Turkey
Estonia
Tajikistan
Comoros
Djibouti
Eritrea
United Arab Emirates
Seychelles
Uzbekistan
Oman
Swaziland
Iraq
Kuwait
Chad
Equatorial Guinea
Bahrain
Jordan
Qatar
Saudi Arabia
Lesotho
Bermuda
Dominican Republic
Argentina
Belize
Uruguay
Georgia
Guinea
Burkina Faso
Albania
Hungary
Cameroon
Cyprus
Denmark
Greece
Netherlands
Belgium
Poland
Ukraine
France
Cameroon
Morocco
Antigua and Barbuda
Suriname
Colombia
Guyana
Paraguay
Saint Lucia
Nicaragua
Guatemala
Somalia
Panama
Mexico
South Africa
Nigeria
Algeria
San Marino
Sudan
Morocco
Turkey
Brazil
Croatia
Montenegro
Ireland
Portugal
Barbados
Puerto Rico
Cayman Islands
Thailand
Singapore
Indonesia
Burundi
Lebanon
Madagascar
Kenya
Uzbekistan
Rwanda
Kuwait
Togo
Jordan
Qatar
Saudi Arabia
Mozambique
Namibia
Grenada
Haiti
Colombia
Ecuador
Honduras
N

Tunisia
Oman
Bahrain
United Arab Emirates
Zambia
South Africa
Iraq
Yemen
Gabon
Niger
Ghana
Bahrain
United Arab Emirates
Panama
Iraq
Kuwait
Zambia
South Africa
Namibia
Panama
Ghana
Monaco
United Arab Emirates
Iraq
Lebanon
Chile
Burkina Faso
Bahrain
Iraq
Guatemala
South Africa
Angola
Honduras
Costa Rica
Ghana
Mali
San Marino
Belize
Chile
Zambia
Nigeria
Costa Rica
Tunisia
Nicaragua
South Africa
Morocco
Panama
Costa Rica
Sweden
Thailand
Ghana
Niger
China
Zambia
Burkina Faso
Guatemala
Algeria
Monaco
San Marino
Honduras
Costa Rica
Canada
Sweden
Morocco
Belize
Niger
Honduras
Burkina Faso
Ethiopia
Algeria
Togo
USA
Oman
Mexico
Jordan
Qatar
Bahrain
Azerbaijan
Iraq
Ghana
South Africa
Poland
Burkina Faso
Luxembourg
Senegal
Mali
Burkina Faso
Uzbekistan
Thailand
Oman
Iraq
Jordan
Yemen
Qatar
Saudi Arabia
Honduras
Nepal
Japan
India
Azerbaijan
Croatia
Kenya
Belarus
Cyprus
Chile
Norway
Slovenia
Albania
Israel
Spain
Turkey
Malta
Greece
Iceland
Netherlands
Sweden
Belgium
Ireland
Romania
France
Portugal
Pa

France
Honduras
Costa Rica
United Arab Emirates
Switzerland
Brazil
Greece
Mexico
Gibraltar
Algeria
Hungary
Netherlands
Italy
Ecuador
Egypt
Iceland
Argentina
Uruguay
Chile
United Arab Emirates
Poland
Germany
Brazil
Colombia
Croatia
Japan
Costa Rica
Greece
Mexico
Cameroon
Estonia
Hungary
Argentina
Belgium
Spain
USA
United Arab Emirates
San Marino
France
Ghana
Portugal
Brazil
Mexico
Spain
Chile
Colombia
Uruguay
Switzerland
France
Argentina
Germany
Ghana
Belgium
Brazil
Australia
Spain
China
Cameroon
Colombia
Uruguay
Japan
Italy
Switzerland
Honduras
Argentina
Germany
Indonesia
Nigeria
Belgium
China
USA
Australia
Netherlands
Cameroon
Croatia
Costa Rica
Italy
Greece
Japan
Nigeria
Ecuador
Honduras
Indonesia
Portugal
USA
Algeria
Brazil
Colombia
Netherlands
Costa Rica
China
France
Germany
Argentina
Belgium
France
Brazil
Argentina
Netherlands
Brazil
Netherlands
Brazil
Germany
Qatar
Botswana
Uganda
Sierra Leone
Lesotho
Congo
Benin
Malawi
Rwanda
Kenya
Mozambique
Peru
Monaco
Kazakhstan
Romania
Azerb

In [28]:
my_df.to_csv('data_odd_match.csv')

In [29]:
my_df.shape

(6747, 19)

In [26]:
my_df = data_odd.iloc[1:3,:]
my_df = my_df[0:0]
my_df

,match_id,league,match_date,home_team,home_score,away_team,away_score,avg_odds_home_win,avg_odds_draw,avg_odds_away_win,max_odds_home_win,max_odds_draw,max_odds_away_win,top_bookie_home_win,top_bookie_draw,top_bookie_away_win,n_odds_home_win,n_odds_draw,n_odds_away_win


In [24]:
length

479440

In [15]:
#  Migrate data
curr_match = data_2005.iloc[30,:]
curr_match

Unnamed: 0                     26974
id                             26974
date                      2005-02-01
team_1                         Haiti
team_2           Trinidad and Tobago
home_team        Trinidad and Tobago
tournament                  Friendly
h_win_diff                        -2
h_draw                             5
f_goalF_1                         22
f_goalF_2                         21
f_goalA_1                         16
f_goalA_2                         10
f_win_1                            4
f_win_2                            7
f_draw_1                           2
f_draw_2                           0
ratio_odds                       NaN
avg_odds_draw                    NaN
result                          lose
Name: 30, dtype: object

In [16]:
data_odd[(data_odd['match_date'] == curr_match.date) & (data_odd['home_team'] == curr_match.home_team)]

,match_id,league,match_date,home_team,home_score,away_team,away_score,avg_odds_home_win,avg_odds_draw,avg_odds_away_win,max_odds_home_win,max_odds_draw,max_odds_away_win,top_bookie_home_win,top_bookie_draw,top_bookie_away_win,n_odds_home_win,n_odds_draw,n_odds_away_win


In [ ]:
import requests

req = requests.get('https://raw.githubusercontent.com/Miguel-Frazao/world-data/master/countries_data.json').json()
countries = (i['name'] for i in req)
print(list(countries))

In [18]:
from iso3166 import countries

In [20]:
'Germany U23' in countries

False

In [ ]:
data_2005['tournament'].unique()

array(['AFF Championship', 'Friendly', 'CFU Caribbean Cup qualification',
       'Cyprus International Tournament', 'FIFA World Cup qualification',
       'Lunar New Year Cup', 'UNCAF Cup', 'CFU Caribbean Cup',
       'COSAFA Cup', 'EAFF Championship', 'Baltic Cup', 'Kirin Cup',
       'Confederations Cup', 'Gold Cup', 'Amílcar Cabral Cup',
       'CECAFA Cup', 'SAFF Cup', 'AFC Asian Cup qualification',
       "King's Cup", 'African Cup of Nations',
       'Malta International Tournament', 'AFC Challenge Cup',
       'FIFA World Cup', 'UEFA Euro qualification', 'Merdeka Tournament',
       'African Cup of Nations qualification', 'Copa del Pacífico',
       'UAFA Cup qualification', 'Gulf Cup', 'WAFF Championship',
       'Copa América', 'AFC Asian Cup', 'Nehru Cup', 'South Pacific Games',
       'AFC Challenge Cup qualification', 'UEFA Euro',
       'African Nations Championship', 'VFF Cup', 'Dragon Cup',
       'ABCS Tournament', 'Nile Basin Tnmt.', 'Nations Cup',
       'Copa Paz del

In [115]:
curr_match

date          2018-03-23
home_team        Germany
away_team          Spain
home_score             1
away_score             1
tournament      Friendly
city          Düsseldorf
country          Germany
id                 38792
winner              None
Name: 38792, dtype: object

In [116]:
team_1 = 'Germany'
team_2 = 'Spain'
df_h2h = data[(((data.home_team == team_1) & (data.away_team == team_2)) | 
            ((data.home_team == team_2) & (data.away_team == team_1))) & 
            (data.date < curr_match.date)]
df_h2h

,date,home_team,away_team,home_score,away_score,tournament,city,country,id,winner
1741,1935-05-12,Germany,Spain,1,2,Friendly,Cologne,Germany,1741,Spain
1822,1936-02-23,Spain,Germany,1,2,Friendly,Barcelona,Spain,1822,Germany
2356,1942-04-12,Germany,Spain,1,1,Friendly,Berlin,Germany,2356,None
3355,1952-12-28,Spain,Germany,2,2,Friendly,Madrid,Spain,3355,None
4248,1958-03-19,Germany,Spain,2,0,Friendly,Frankfurt am Main,Germany,4248,Germany
6163,1966-07-20,Germany,Spain,2,1,FIFA World Cup,Birmingham,England,6163,Germany
7298,1970-02-11,Spain,Germany,2,0,Friendly,Seville,Spain,7298,Spain
8755,1973-11-24,Germany,Spain,2,1,Friendly,Stuttgart,Germany,8755,Germany
8820,1974-02-23,Spain,Germany,1,0,Friendly,Barcelona,Spain,8820,Spain
9607,1976-04-24,Spain,Germany,1,1,UEFA Euro qualification,Madrid,Spain,9607,None


In [112]:
list(range(3))

[0, 1, 2]

In [110]:
# How many data we throw
1 - data_2005.shape[0] / data.shape[0]
# we throw 70% of data

0.6926122050280191

In [16]:
# First filter data from 2005
# 2005-01-01 to 2015-07-01
# For example filter data of Germany vs Spain
df1 = data[(data.home_team == 'Spain') & (data.away_team == 'Germany')]
df2 = data[(data.home_team == 'Germany') & (data.away_team == 'Spain')]
df3 = df1.append(df2).sort_values(by=['date'])
df3.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,id
1741,1935-05-12,Germany,Spain,1,2,Friendly,Cologne,Germany,1741
1822,1936-02-23,Spain,Germany,1,2,Friendly,Barcelona,Spain,1822
2356,1942-04-12,Germany,Spain,1,1,Friendly,Berlin,Germany,2356
3355,1952-12-28,Spain,Germany,2,2,Friendly,Madrid,Spain,3355
4248,1958-03-19,Germany,Spain,2,0,Friendly,Frankfurt am Main,Germany,4248


In [62]:
df3

,date,home_team,away_team,home_score,away_score,tournament,city,country,id
1741,1935-05-12,Germany,Spain,1,2,Friendly,Cologne,Germany,1741
1822,1936-02-23,Spain,Germany,1,2,Friendly,Barcelona,Spain,1822
2356,1942-04-12,Germany,Spain,1,1,Friendly,Berlin,Germany,2356
3355,1952-12-28,Spain,Germany,2,2,Friendly,Madrid,Spain,3355
4248,1958-03-19,Germany,Spain,2,0,Friendly,Frankfurt am Main,Germany,4248
6163,1966-07-20,Germany,Spain,2,1,FIFA World Cup,Birmingham,England,6163
7298,1970-02-11,Spain,Germany,2,0,Friendly,Seville,Spain,7298
8755,1973-11-24,Germany,Spain,2,1,Friendly,Stuttgart,Germany,8755
8820,1974-02-23,Spain,Germany,1,0,Friendly,Barcelona,Spain,8820
9607,1976-04-24,Spain,Germany,1,1,UEFA Euro qualification,Madrid,Spain,9607


In [84]:
# define our own datafram
col_names = ['id','date', 'team_1', 'team_2', 'home_team', 'tournament', 'h_win_diff', 'h_draw', 'f_goalF_1', 
             'f_goalF_2','f_goalA_1','f_goalA_2', 'f_win_1', 'f_win_2', 'f_draw_1', 'f_draw_2','result']
my_df = pd.DataFrame(columns = col_names)
my_df

,id,date,team_1,team_2,home_team,tournament,h_win_diff,h_draw,f_goalF_1,f_goalF_2,f_goalA_1,f_goalA_2,f_win_1,f_win_2,f_draw_1,f_draw_2,result


In [18]:
# Now process in df3 to get desired information
# For example FIFA world cup 2010
df_test = df3.copy()
# Add new column = result to df3
df_test['winner'] = np.where(df_test.home_score > df_test.away_score, df_test.home_team, 
                                 np.where(df_test.home_score < df_test.away_score, df_test.away_team, 'None'))
df_test.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,id,winner
1741,1935-05-12,Germany,Spain,1,2,Friendly,Cologne,Germany,1741,Spain
1822,1936-02-23,Spain,Germany,1,2,Friendly,Barcelona,Spain,1822,Germany
2356,1942-04-12,Germany,Spain,1,1,Friendly,Berlin,Germany,2356,None
3355,1952-12-28,Spain,Germany,2,2,Friendly,Madrid,Spain,3355,None
4248,1958-03-19,Germany,Spain,2,0,Friendly,Frankfurt am Main,Germany,4248,Germany


In [ ]:
# Get ten recent matches - sort and take 10
# df_10 = data[data[home_team] == 'Germany' or ]


date          2008-06-29
home_team        Germany
away_team          Spain
home_score             0
away_score             1
tournament     UEFA Euro
city              Vienna
country          Austria
winner             Spain
Name: 30052, dtype: object

In [19]:
curr_match = df_test.iloc[-4, :]
curr_match

date          2008-06-29
home_team        Germany
away_team          Spain
home_score             0
away_score             1
tournament     UEFA Euro
city              Vienna
country          Austria
id                 30052
winner             Spain
Name: 30052, dtype: object

In [20]:
df_match = df_test[df_test['date'] < curr_match['date'] ]
df_match.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,id,winner
1741,1935-05-12,Germany,Spain,1,2,Friendly,Cologne,Germany,1741,Spain
1822,1936-02-23,Spain,Germany,1,2,Friendly,Barcelona,Spain,1822,Germany
2356,1942-04-12,Germany,Spain,1,1,Friendly,Berlin,Germany,2356,None
3355,1952-12-28,Spain,Germany,2,2,Friendly,Madrid,Spain,3355,None
4248,1958-03-19,Germany,Spain,2,0,Friendly,Frankfurt am Main,Germany,4248,Germany


In [31]:
%config IPCompleter.greedy=True

In [27]:
# Now we do history analysis here
from collections import Counter
count = Counter(df_match['winner']) #Now we get no.win and no.draw
count

Counter({'Germany': 8, 'None': 6, 'Spain': 5})

In [57]:
count['Germany']

8

In [58]:
# Now fill in info in our table
# col_names = ['id','date', 'team_1', 'team_2', 'home_team', 'tournament', 'h_win_diff', 'h_draw', 'f_num_goal_1', 
#              'f_num_goal_2', 'f_win_1', 'f_win_2', 'f_draw_1', 'f_draw_2']
# pre_df = pd.DataFrame(columns = col_names)
# my_df

pre_df = curr_match[['id','date','tournament']].copy()
# 
pre_df['team_1'] =  curr_match['home_team'] if curr_match['home_team'] < curr_match['away_team'] else curr_match['away_team'] 
pre_df['team_2'] =  curr_match['away_team'] if curr_match['home_team'] < curr_match['away_team'] else curr_match['away_team'] 
pre_df['home_team'] = curr_match['home_team']

pre_df['h_win_diff'] = count[pre_df['team_1']] - count[pre_df['team_2']]
pre_df['h_draw'] = count['None']


In [104]:
# Assemble
# Before this loop, I need to add winner here
import time

# Drop all data
my_df = my_df[0:0]

# need to get df_test here

start_time = time.time()
# Test history match stat
for i in range(-4,0):
    print(i)
    curr_match = df_test.iloc[i,:]
    df_match = df_test[df_test['date'] < curr_match['date'] ]
    count = Counter(df_match['winner']) #Now we get no.win and no.draw
    
    pre_df = curr_match[['id','date','tournament']].copy()
    
#     	id	date	team_1	team_2	home_team	tournament	h_win_diff	h_draw	f_goalF_1	f_goalF_2	f_goalA_1	f_goalA_2	f_win_1	f_win_2	f_draw_1	f_draw_2	result
    # Get the head-to-head result
    pre_df['team_1'] =  curr_match['home_team'] if curr_match['home_team'] < curr_match['away_team'] else curr_match['away_team'] 
    pre_df['team_2'] =  curr_match['away_team'] if curr_match['home_team'] < curr_match['away_team'] else curr_match['home_team'] 
    pre_df['home_team'] = curr_match['home_team']

    pre_df['h_win_diff'] = count[pre_df['team_1']] - count[pre_df['team_2']]
    pre_df['h_draw'] = count['None']
    
    # Get the recent result of each team
    # Team 1
    team_1 = pre_df['team_1']
    df_10  = data[(data.home_team == team_1) | (data.away_team == team_1)]
    
    length = df_10.shape[0]
    df_10  = df_10.iloc[length-11:length-1,:]
    
    df_10['winner'] = np.where(df_10.home_score > df_10.away_score, df_10.home_team, 
                                     np.where(df_10.home_score < df_10.away_score, df_10.away_team, 'None'))
    
    pre_df['f_goalF_1'] = df_10['home_score'].sum()
    pre_df['f_goalA_1'] = df_10['away_score'].sum()
    
    dict_result_10 = Counter(df_10['winner'])
    pre_df['f_win_1']  = dict_result_10[team_1]
    pre_df['f_draw_1'] = dict_result_10['None']
    
    # Team 2
    team_2 = pre_df['team_2']
    df_10  = data[(data.home_team == team_2) | (data.away_team == team_2)]
    
    length = df_10.shape[0]
    df_10  = df_10.iloc[length-11:length-1,:]
    
    df_10['winner'] = np.where(df_10.home_score > df_10.away_score, df_10.home_team, 
                                     np.where(df_10.home_score < df_10.away_score, df_10.away_team, 'None'))
    
    pre_df['f_goalF_2'] = df_10['home_score'].sum()
    pre_df['f_goalA_2'] = df_10['away_score'].sum()
    
    dict_result_10 = Counter(df_10['winner'])
    pre_df['f_win_2']  = dict_result_10[team_2]
    pre_df['f_draw_2'] = dict_result_10['None']
    
    pre_df['result'] = np.where(curr_match.winner == team_1, 'win', 
                                     np.where(curr_match.winner == team_2, 'lose', 'draw'))
    
    
    my_df = my_df.append(pre_df)
    
print(time.time() - start_time)

-4
-3
-2
-1
2.9774169921875


In [93]:
#  Ok now move to the next step: retrieve the recent performance 
team_1 = curr_match['home_team']

df_10  = data[(data.home_team == team_1) | (data.away_team == team_1)]
length = df_10.shape[0]
df_10  = df_10.iloc[length-11:length-1,:]
df_10['winner'] = np.where(df_10.home_score > df_10.away_score, df_10.home_team, 
                                 np.where(df_10.home_score < df_10.away_score, df_10.away_team, 'None'))
df_10

# df2 = data[(data.home_team == 'Germany') & (data.away_team == 'Spain')]
# df3 = df1.append(df2).sort_values(by=['date'])
# df_10.iloc[:]

,date,home_team,away_team,home_score,away_score,tournament,city,country,id,winner
38256,2017-06-25,Germany,Cameroon,3,1,Confederations Cup,Sochi,Russia,38256,Germany
38268,2017-06-29,Germany,Mexico,4,1,Confederations Cup,Sochi,Russia,38268,Germany
38279,2017-07-02,Chile,Germany,0,1,Confederations Cup,St Petersburg,Russia,38279,Germany
38378,2017-09-01,Czech Republic,Germany,1,2,FIFA World Cup qualification,Prague,Czech Republic,38378,Germany
38415,2017-09-04,Germany,Norway,6,0,FIFA World Cup qualification,Stuttgart,Germany,38415,Germany
38476,2017-10-05,Northern Ireland,Germany,1,3,FIFA World Cup qualification,Belfast,Northern Ireland,38476,Germany
38528,2017-10-08,Germany,Azerbaijan,5,1,FIFA World Cup qualification,Kaiserslautern,Germany,38528,Germany
38609,2017-11-10,England,Germany,0,0,Friendly,London,England,38609,None
38678,2017-11-14,Germany,France,2,2,Friendly,Cologne,Germany,38678,None
38792,2018-03-23,Germany,Spain,1,1,Friendly,Düsseldorf,Germany,38792,None
